In [18]:
import os
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv('50k_data.csv')

In [4]:
df

,Unnamed: 0,ImageID,OriginalURL,OriginalSize,OriginalMD5,tags
0,0,4fa8054781a4c382,https://farm3.staticflickr.com/5310/5898076654...,4405052,KFukvivpCM5QXl5SqKe41g==,"['Clock', 'House', 'Flower', 'Person', 'Clothi..."
1,1,d05c3e451f79174d,https://c6.staticflickr.com/8/7375/13791738583...,972558,KoVMrVWKftzadg1EGr6nCg==,['Pineapple lumps']
2,2,5bde9ad21e82deed,https://c6.staticflickr.com/1/223/495658260_89...,234199,dsfj7rNAbGq67OzPRAGdsQ==,"['Lake', 'Mountain']"
3,3,b37f763ae67d0888,https://c1.staticflickr.com/1/67/197493648_628...,494555,9IzEn38GRNsVpATuv7gzEA==,"['Golf', 'Person', 'Man', 'Plant', 'Clothing']"
4,4,3231e75bd7120b7a,https://c3.staticflickr.com/4/3654/3619297168_...,2570210,195AcCHMqin3MYTv+pTDuA==,"['Food', 'Vodka red bull']"
...,...,...,...,...,...,...
49995,49995,87fd0345f04f54b5,https://farm6.staticflickr.com/8314/7999226458...,321064,O9AJBWCmaaJQNjz2i7mCuw==,"['Book', 'Handwriting']"
49996,49996,78ffd2a4640e5226,https://farm7.staticflickr.com/5323/6914715410...,685696,iMSjdKeilWwrMhAls1RsKg==,"['Clock', 'Person', 'Chair', 'Suit', 'Couch', ..."
49997,49997,653a8f435b3e8440,https://farm1.staticflickr.com/3073/5871264695...,1879421,l3Tq+WIt1IRiiB4Vf97tGw==,['Dance']
49998,49998,080c273ba6f119f3,https://farm4.staticflickr.com/5337/1777916298...,13625931,HeOJePneWEhK5Twd+c+VhQ==,"['Boxing', 'Quad skates', 'Amateur boxing', 'B..."


In [23]:
files=os.listdir('features-20231107T171535Z-001/features')

In [24]:
features=[]

In [25]:
for filename in files:
    id=file.split('.')[0]
    feature_path = os.path.join('features-20231107T171535Z-001/features', filename)
    feature=np.load(feature_path)
    features.append(feature)

In [26]:
from sklearn.decomposition import PCA
import joblib

features_array = np.vstack(features)

# Initialize PCA to reduce dimensions to 795
pca = PCA(n_components=795)

# Fit PCA on the entire dataset and transform the features
pca.fit(features_array)

# Save the trained PCA model for later use
joblib.dump(pca, 'best_pca_model.pkl')

['best_pca_model.pkl']

In [29]:
np.save('40k_features.npy',features_array)

In [35]:
df[df['ImageID'] == '3231e75bd7120b7a']

,Unnamed: 0,ImageID,OriginalURL,OriginalSize,OriginalMD5,tags
4,4,3231e75bd7120b7a,https://c3.staticflickr.com/4/3654/3619297168_...,2570210,195AcCHMqin3MYTv+pTDuA==,"['Food', 'Vodka red bull']"


In [42]:
from elasticsearch import Elasticsearch


es = Elasticsearch(["http://localhost:9200"])
# index = "image_features"
index = 'features'

In [30]:
def reduce_features(np_file):
    file=np.load(np_file)
    if file.ndim == 1:
        file = file.reshape(1, -1)
    reduced_features = pca.transform(file)
    return reduced_features[0]

In [32]:
def upload(image_path,features,tags):
    # Create a document for the image
    doc = {
        
        "featurevector": features.tolist(),  # Convert NumPy array to list
        "imgUrl": image_path,
        "tags": tags
        # Add other fields if necessary
    }
    
    # Index the document
    res = es.index(index=index, document=doc)
    print(res['result'], res['_id'])

In [43]:
for filename in files:
    id=filename.split('.')[0]
    row=df[df['ImageID'] == id]
    feature_path = os.path.join('features-20231107T171535Z-001/features', filename)
    reduced_feature=reduce_features(feature_path)
    upload(row['OriginalURL'].values[0],reduced_feature,row['tags'].values[0])
    
    

created 4Tirq4sBluYwGNyFo5H6
created 4jirq4sBluYwGNyFppHX
created 4zirq4sBluYwGNyFp5FM
created 5Dirq4sBluYwGNyFqJE6
created 5Tirq4sBluYwGNyFqZGR
created 5jirq4sBluYwGNyFqpGL
created 5zirq4sBluYwGNyFq5G_
created 6Dirq4sBluYwGNyFrJGy
created 6Tirq4sBluYwGNyFrZGS
created 6jirq4sBluYwGNyFrpFw
created 6zirq4sBluYwGNyFr5Fl
created 7Dirq4sBluYwGNyFsJFP
created 7Tirq4sBluYwGNyFsZE5
created 7jirq4sBluYwGNyFspEk
created 7zirq4sBluYwGNyFs5EP
created 8Dirq4sBluYwGNyFs5Hu
created 8Tirq4sBluYwGNyFtJHM
created 8jirq4sBluYwGNyFtZHb
created 8zirq4sBluYwGNyFt5FC
created 9Dirq4sBluYwGNyFuJEz
created 9Tirq4sBluYwGNyFuZEe
created 9jirq4sBluYwGNyFupEJ
created 9zirq4sBluYwGNyFupH0
created -Dirq4sBluYwGNyFu5HT
created -Tirq4sBluYwGNyFvJG9
created -jirq4sBluYwGNyFvZG9
created -zirq4sBluYwGNyFvpGm
created _Dirq4sBluYwGNyFv5H0
created _Tirq4sBluYwGNyFwZEt
created _jirq4sBluYwGNyFwpGj
created _zirq4sBluYwGNyFw5Gw
created ADirq4sBluYwGNyFxJKy
created ATirq4sBluYwGNyFxZKc
created Ajirq4sBluYwGNyFxpKG
created Azirq4